## This notebook is the final project of ibm data science specialization. Here I consider a business problem of building an electric vehicle charging station in Moscow.

_Let's import the necessary libraries._

In [1]:
import numpy as np
import pandas as pd
#import xlrd
import sklearn
import requests
from sklearn.cluster import KMeans
#!conda install folium -c conda-forge
import folium

In [2]:
evs=pd.read_csv('data-25672-2019-06-18_EN.csv',sep=';', comment='#')

_Now we will create a dataframe with location of electric vehicle stations(evs)._

In [3]:
evs

,ID_en,Name_en,AdmArea_en,District_en,Address_en,Longitude_WGS84_en,Latitude_WGS84_en,global_id,geoData,Unnamed: 9
0,1,"ЭЗС улица Бахрушина, дом 20",Czentral`ny'j administrativny'j okrug,rajon Zamoskvorech`e,"улица Бахрушина, дом 20",37.635950,55.734389,58937156,"{type=Point, coordinates=[37.635949800384, 55....",NaN
1,7,"ЭЗС Глинищевский переулок, дом 6",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"город Москва, Глинищевский переулок, дом 6",37.609815,55.763042,282292364,"{type=Point, coordinates=[37.609815, 55.763042]}",NaN
2,8,"ЭЗС улица Малая Дмитровка, дом 15",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"улица Малая Дмитровка, дом 15",37.605320,55.770210,282292416,"{type=Point, coordinates=[37.605319999837, 55....",NaN
3,9,"ЭЗС Газетный переулок, дом 1/12",Czentral`ny'j administrativny'j okrug,Presnenskij rajon,"Газетный переулок, дом 1/12",37.607530,55.756691,282292417,"{type=Point, coordinates=[37.607530000065, 55....",NaN
4,13,"ЭЗС Большой Николоворобинский переулок, дом 7,...",Czentral`ny'j administrativny'j okrug,Taganskij rajon,"город Москва, Большой Николоворобинский переул...",37.648119,55.751665,282292370,"{type=Point, coordinates=[37.648119, 55.751665]}",NaN
5,14,"ЭЗС улица Остоженка, дом 36",Czentral`ny'j administrativny'j okrug,rajon Xamovniki,"город Москва, улица Остоженка, дом 36",37.595418,55.738996,282292371,"{type=Point, coordinates=[37.595418, 55.738996]}",NaN
6,16,"ЭЗС Леонтьевский переулок, дом 25(1)",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"Леонтьевский переулок, дом 25",37.607225,55.761974,282292423,"{type=Point, coordinates=[37.607224999568, 55....",NaN
7,17,"ЭЗС Леонтьевский переулок, дом 25(2)",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"Леонтьевский переулок, дом 25",37.607197,55.761954,282292385,"{type=Point, coordinates=[37.607196999979, 55....",NaN
8,19,"ЭЗС улица Большая Бронная, дом 14",Czentral`ny'j administrativny'j okrug,Presnenskij rajon,"город Москва, Большая Бронная улица, дом 14, с...",37.600217,55.762402,282292441,"{type=Point, coordinates=[37.600217, 55.762402]}",NaN
9,20,"ЭЗС Мароновский переулок, дом 1/7",Czentral`ny'j administrativny'j okrug,rajon Yakimanka,"город Москва, Мароновский переулок, дом 1/7, с...",37.610525,55.735565,282292438,"{type=Point, coordinates=[37.610525, 55.735565]}",NaN


In [4]:
evs.shape

(35, 10)

_Let's check how many rows are there and edit columns._

In [5]:
del evs['Unnamed: 9']

In [6]:
evs=evs.drop([34])

In [7]:
evs.rename(columns={'Address_en':'Address','Longitude_WGS84_en':'Longitude','Latitude_WGS84_en':'Latitude'},inplace=True)

_The latitude of Moscow, Russia is 55.751244, and the longitude is 37.618423._

In [8]:
mow_lat=55.751244
mow_long=37.618423

_Let's create a map of moscow with evs as markers._

In [9]:
map_moscow = folium.Map(location=[mow_lat, mow_long], zoom_start=11)

# add markers to map
for lat, lng, address, district in zip(evs['Latitude'], evs['Longitude'], evs['Address'], evs['District_en']):
    label = '{}, {}'.format(district, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)  
    
map_moscow

In [10]:
CLIENT_ID = 'JDCWCBXAEIEFXHTFASSXP204MAHTSWKMBWJMK2GX1NI1CBWL' 
CLIENT_SECRET = 'DBC1IQFBVBETNRRXLAGBP2MP33I1GUYZEXSBC3TURA4PZVTS' 
VERSION = '20180605' 

_Let's explore top venues near each evs._

In [11]:
# defining radius and limit of venues to get
radius=250
LIMIT=100

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Evs', 
                  'Evs Latitude', 
                  'Evs Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
moscow_venues = getNearbyVenues(names=evs['Name_en'],
                                   latitudes=evs['Latitude'],
                                   longitudes=evs['Longitude']
                                  )

In [14]:
moscow_venues.head(10)

,Evs,Evs Latitude,Evs Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Mercure Paveletskaya,55.735884,37.635181,Hotel
1,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Moments,55.733434,37.638830,Gastropub
2,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Bro.we,55.734545,37.638030,Coffee Shop
3,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Ibis,55.735838,37.635767,Hotel
4,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Adagio Paveletskaya,55.736228,37.635772,Hotel
5,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Фирменный магазин ОАО «Рот-Фронт»,55.734636,37.632013,Candy Store
6,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Пять звёзд,55.732922,37.637075,Multiplex
7,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Dada,55.734861,37.632620,Caucasian Restaurant
8,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Скверик На Новокузнецкой,55.734265,37.633849,Park
9,"ЭЗС улица Бахрушина, дом 20",55.734389,37.63595,Двор,55.733835,37.638428,Playground


In [15]:
moscow_venues.shape

(670, 7)

In [16]:
for cat,lat,long in zip(moscow_venues['Venue Category'], moscow_venues['Venue Latitude'], moscow_venues['Venue Longitude'],):
    label = '{}'.format(cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow) 

_Here is a map with venues._

In [17]:
map_moscow

In [18]:
moscow_venues.groupby('Venue Category').count()

,Evs,Evs Latitude,Evs Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,1,1,1,1,1,1
Antique Shop,2,2,2,2,2,2
Arcade,2,2,2,2,2,2
Art Gallery,13,13,13,13,13,13
Arts & Crafts Store,8,8,8,8,8,8
Asian Restaurant,4,4,4,4,4,4
Baby Store,3,3,3,3,3,3
Bakery,19,19,19,19,19,19
Bar,11,11,11,11,11,11


In [19]:
items_counts = moscow_venues['Venue Category'].value_counts(normalize=True)
items_counts

Coffee Shop                                 0.077612
Café                                        0.038806
Restaurant                                  0.034328
Bakery                                      0.028358
Bookstore                                   0.026866
                                              ...   
Residential Building (Apartment / Condo)    0.001493
Deli / Bodega                               0.001493
Shawarma Place                              0.001493
Czech Restaurant                            0.001493
Hobby Shop                                  0.001493
Name: Venue Category, Length: 154, dtype: float64

_Now we will find out which categories are more common._

In [20]:
# one hot encoding
moscow_onehot = pd.get_dummies(moscow_venues[['Venue Category']], prefix="", prefix_sep="")

# add evs column back to dataframe
moscow_onehot['Evs'] = moscow_venues['Evs'] 

# move evs column to the first column
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]

moscow_onehot.head()

,Evs,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Baby Store,Bakery,Bar,...,Toy / Game Store,Track,University,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"ЭЗС улица Бахрушина, дом 20",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"ЭЗС улица Бахрушина, дом 20",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"ЭЗС улица Бахрушина, дом 20",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"ЭЗС улица Бахрушина, дом 20",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"ЭЗС улица Бахрушина, дом 20",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


_Let's apply K mean clustering to our data. But first we need to prepare it._

In [21]:
moscow_venues_grouped = moscow_onehot.groupby('Evs').mean().reset_index()

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
#Now let's create the new dataframe and display the top 10 venues for each evs
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Evs']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
evs_venues_sorted = pd.DataFrame(columns=columns)
evs_venues_sorted['Evs'] = moscow_venues_grouped['Evs']

for ind in np.arange(moscow_venues_grouped.shape[0]):
    evs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_venues_grouped.iloc[ind, :], num_top_venues)

In [24]:
#Let's assume based on our observations that there are 4 clusters
kclusters = 4

moscow_grouped_clustering = moscow_venues_grouped.drop('Evs', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 2, 1, 2, 3, 0, 0, 1], dtype=int32)

In [25]:
# add clustering labels
evs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

moscow_merged = evs

moscow_merged = moscow_merged.join(evs_venues_sorted.set_index('Evs'), on='Name_en')

moscow_merged.head()

,ID_en,Name_en,AdmArea_en,District_en,Address,Longitude,Latitude,global_id,geoData,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"ЭЗС улица Бахрушина, дом 20",Czentral`ny'j administrativny'j okrug,rajon Zamoskvorech`e,"улица Бахрушина, дом 20",37.635950,55.734389,58937156,"{type=Point, coordinates=[37.635949800384, 55....",2,Hotel,Park,Hookah Bar,Coffee Shop,Caucasian Restaurant,Café,Modern European Restaurant,Gastropub,Playground,Museum
1,7,"ЭЗС Глинищевский переулок, дом 6",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"город Москва, Глинищевский переулок, дом 6",37.609815,55.763042,282292364,"{type=Point, coordinates=[37.609815, 55.763042]}",1,Beer Bar,Clothing Store,Bookstore,Hostel,Pedestrian Plaza,Coffee Shop,Pharmacy,Pizza Place,Plaza,Pub
2,8,"ЭЗС улица Малая Дмитровка, дом 15",Czentral`ny'j administrativny'j okrug,Tverskoj rajon,"улица Малая Дмитровка, дом 15",37.605320,55.770210,282292416,"{type=Point, coordinates=[37.605319999837, 55....",0,Theater,Convenience Store,Fountain,Gaming Cafe,Music Venue,Public Art,Belgian Restaurant,Steakhouse,Bed & Breakfast,Tea Room
3,9,"ЭЗС Газетный переулок, дом 1/12",Czentral`ny'j administrativny'j okrug,Presnenskij rajon,"Газетный переулок, дом 1/12",37.607530,55.756691,282292417,"{type=Point, coordinates=[37.607530000065, 55....",0,Concert Hall,Gym / Fitness Center,Cocktail Bar,Restaurant,Park,Science Museum,Boutique,Spa,Bistro,Coffee Shop
4,13,"ЭЗС Большой Николоворобинский переулок, дом 7,...",Czentral`ny'j administrativny'j okrug,Taganskij rajon,"город Москва, Большой Николоворобинский переул...",37.648119,55.751665,282292370,"{type=Point, coordinates=[37.648119, 55.751665]}",2,Bakery,Historic Site,Karaoke Bar,Residential Building (Apartment / Condo),Café,Eastern European Restaurant,Bed & Breakfast,Caucasian Restaurant,Martial Arts Dojo,Spa


_Let's take a look at our clusters on the map._

In [26]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[mow_lat, mow_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitude'], moscow_merged['Longitude'], moscow_merged['Name_en'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
del moscow_merged['geoData']
del moscow_merged['global_id']

_Let's examine clusters._

In [28]:
#Cluster 0
moscow_merged.loc[moscow_merged['Cluster Labels'] == 0, moscow_merged.columns[[1] + list(range(5, moscow_merged.shape[1]))]]

,Name_en,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"ЭЗС улица Малая Дмитровка, дом 15",37.605320,55.770210,0,Theater,Convenience Store,Fountain,Gaming Cafe,Music Venue,Public Art,Belgian Restaurant,Steakhouse,Bed & Breakfast,Tea Room
3,"ЭЗС Газетный переулок, дом 1/12",37.607530,55.756691,0,Concert Hall,Gym / Fitness Center,Cocktail Bar,Restaurant,Park,Science Museum,Boutique,Spa,Bistro,Coffee Shop
15,ЭЗС улица Александра Солженицына №1,37.663031,55.744164,0,Coffee Shop,Garden,Theater,Health Food Store,Gastropub,Nightclub,Dance Studio,Restaurant,Sandwich Place,Spa
16,ЭЗС улица Александра Солженицына №2,37.663080,55.744182,0,Coffee Shop,Garden,Theater,Health Food Store,Gastropub,Nightclub,Dance Studio,Restaurant,Sandwich Place,Spa
17,ЭЗС 1-й Голутвинский переулок,37.613719,55.739250,0,Restaurant,Café,Nightclub,Mediterranean Restaurant,Bookstore,Pastry Shop,Concert Hall,Coffee Shop,Cocktail Bar,Pub
27,"ЭЗС Восточная улица, дом 11",37.658756,55.712808,0,Dance Studio,Shopping Mall,Restaurant,Business Service,Hookah Bar,Fast Food Restaurant,Baby Store,Blini House,Sporting Goods Shop,Theater
30,"ЭЗС 2-й Обыденский переулок, дом 1А",37.604923,55.741622,0,Dance Studio,Art Gallery,Restaurant,Historic Site,French Restaurant,Yoga Studio,Fabric Shop,Food & Drink Shop,Flower Shop,Fast Food Restaurant
31,"ЭЗС Курсовой переулок, дом 4",37.605815,55.741978,0,Dance Studio,Restaurant,Coffee Shop,Cocktail Bar,French Restaurant,Boutique,Snack Place,Salon / Barbershop,Chinese Restaurant,Beer Bar


In [29]:
#Cluster 1
moscow_merged.loc[moscow_merged['Cluster Labels'] == 1, moscow_merged.columns[[1] + list(range(5, moscow_merged.shape[1]))]]

,Name_en,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"ЭЗС Глинищевский переулок, дом 6",37.609815,55.763042,1,Beer Bar,Clothing Store,Bookstore,Hostel,Pedestrian Plaza,Coffee Shop,Pharmacy,Pizza Place,Plaza,Pub
6,"ЭЗС Леонтьевский переулок, дом 25(1)",37.607225,55.761974,1,Beer Bar,Bookstore,Restaurant,Pizza Place,Salon / Barbershop,Café,Fountain,Caucasian Restaurant,Lebanese Restaurant,Clothing Store
7,"ЭЗС Леонтьевский переулок, дом 25(2)",37.607197,55.761954,1,Beer Bar,Bookstore,Restaurant,Pizza Place,Salon / Barbershop,Café,Fountain,Caucasian Restaurant,Lebanese Restaurant,Clothing Store
8,"ЭЗС улица Большая Бронная, дом 14",37.600217,55.762402,1,Theater,Italian Restaurant,Coffee Shop,Jewelry Store,Toy / Game Store,Men's Store,Road,Restaurant,Cocktail Bar,Playground
9,"ЭЗС Мароновский переулок, дом 1/7",37.610525,55.735565,1,Coffee Shop,Supermarket,Sculpture Garden,Vietnamese Restaurant,Park,Gaming Cafe,Salon / Barbershop,Candy Store,Nail Salon,Yoga Studio
10,ЭЗС улица Серпуховский Вал №1,37.619830,55.711677,1,Caucasian Restaurant,Pizza Place,Soup Place,Ice Cream Shop,Indian Restaurant,Health Food Store,Greek Restaurant,Flower Shop,Farmers Market,Falafel Restaurant
11,ЭЗС улица Серпуховский Вал №2,37.619697,55.711679,1,Caucasian Restaurant,Pizza Place,Soup Place,Ice Cream Shop,Indian Restaurant,Health Food Store,Greek Restaurant,Flower Shop,Farmers Market,Falafel Restaurant
12,ЭЗС Автозаводская улица,37.659885,55.707868,1,Arts & Crafts Store,Coffee Shop,Fabric Shop,Beer Store,Mobile Phone Shop,Shawarma Place,Sporting Goods Shop,Pedestrian Plaza,Paper / Office Supplies Store,Pet Store
13,ЭЗС улица Макаренко,37.646220,55.761024,1,Coffee Shop,Restaurant,Bakery,Beer Bar,Performing Arts Venue,Café,Lake,Fountain,Martial Arts Dojo,Mediterranean Restaurant
19,"ЭЗС улица Большая Полянка, дом 28",37.619600,55.735781,1,Café,Yoga Studio,Gym / Fitness Center,Bookstore,Coffee Shop,Bakery,Restaurant,Food & Drink Shop,Flower Shop,Fast Food Restaurant


In [30]:
#Cluster 2
moscow_merged.loc[moscow_merged['Cluster Labels'] == 2, moscow_merged.columns[[1] + list(range(5, moscow_merged.shape[1]))]]

,Name_en,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"ЭЗС улица Бахрушина, дом 20",37.635950,55.734389,2,Hotel,Park,Hookah Bar,Coffee Shop,Caucasian Restaurant,Café,Modern European Restaurant,Gastropub,Playground,Museum
4,"ЭЗС Большой Николоворобинский переулок, дом 7,...",37.648119,55.751665,2,Bakery,Historic Site,Karaoke Bar,Residential Building (Apartment / Condo),Café,Eastern European Restaurant,Bed & Breakfast,Caucasian Restaurant,Martial Arts Dojo,Spa
5,"ЭЗС улица Остоженка, дом 36",37.595418,55.738996,2,Pharmacy,Hotel,Wine Bar,Art Gallery,Burger Joint,Coffee Shop,Italian Restaurant,Café,Salon / Barbershop,Sandwich Place
18,"ЭЗС Гончарная улица, дом 20/1, строение 1",37.648308,55.744266,2,Cocktail Bar,Arcade,Vegetarian / Vegan Restaurant,Park,Italian Restaurant,Bakery,Bar,Seafood Restaurant,Museum,Fabric Shop
22,"ЭЗС Большая Татарская улица, дом 26",37.633512,55.739334,2,Yoga Studio,Bakery,Dance Studio,Coffee Shop,Clothing Store,Candy Store,Gourmet Shop,Thrift / Vintage Store,Bookstore,Track
32,"ЭЗС Старомонетный переулок, дом 11",37.618132,55.741164,2,Art Gallery,Hotel,Irish Pub,Cosmetics Shop,Movie Theater,Salon / Barbershop,Road,Pub,Pool,Middle Eastern Restaurant
33,"ЭЗС Якиманский переулок, дом 6",37.609686,55.733764,2,Boxing Gym,Pool,Salon / Barbershop,Nail Salon,Japanese Restaurant,Bridal Shop,Art Gallery,Arts & Crafts Store,Dessert Shop,Farmers Market


In [31]:
#Cluster 3
moscow_merged.loc[moscow_merged['Cluster Labels'] == 3, moscow_merged.columns[[1] + list(range(5, moscow_merged.shape[1]))]]

,Name_en,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,ЭЗС Вековая улица,37.674578,55.74364,3,Museum,Gift Shop,Baby Store,Park,Theater,Yoga Studio,Electronics Store,Flower Shop,Fast Food Restaurant,Farmers Market
